# Keras Models - CNN with 3 Convolutional Layers and RNN with 2 GRU Layers

Keras is the deep learning framework which is simple to use. I found it easier than to build a new model in torch, so I wanted to use and get results with Keras models.

I tried 5 Keras models for this project. First 2 models can be found in this notebook. Other 3 models were run in Google Colab to get more fast results. So, they can be found in next notebook (number 6 notebook). This gives me a chance to run different models at the same time.

### Aim of This Notebook:

In this notebook, my aim is to get predictions with using Convolutional Neural Net models and Recurrent Neural Net models.

In [ ]:
# dataframe imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import re

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

#tensorflow imports for keras
import tensorflow
from tensorflow.python.keras import models, layers, optimizers
from tensorflow.python.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
df = pd.read_csv('train.csv') # taking data

In [ ]:
df.head() # to check

,review_clean,sentiment
0,i am shocked harrison at the very end gives p...,1
1,the best self help book ive ever read half of ...,1
2,quite interesting a time of intrigue and excit...,1
3,i love the bibliophile series i saw that a eb...,1
4,this is a really great story filled with wonde...,1


In [ ]:
df.dropna(inplace=True) # last more cleaning to make sure for null values

# Splitting Data to Train and Test

To make sure about using same sample data in each notebook, I always get same data and divide with same random state and test_size for validation.

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2,random_state = 42)

# Data Preparation for Keras

I will divide text and target to prepare data to model. I will do it for both train and test set.

In [ ]:
train_target = train_data.sentiment
train_texts = train_data.review_clean

test_target = test_data.sentiment
test_texts = test_data.review_clean

After here, I got inspired from this notebook which is the solution of one Kaggle compatition below,

https://www.kaggle.com/muonneutrino/sentiment-analysis-with-amazon-reviews

I used steps in this notebook to get baseline for Keras and I changed layers types and layers numbers to get better results.

In [ ]:
# I get together my text
def converting_texts(texts):
    collected_texts = []
    for text in texts:
        collected_texts.append(text)
    return collected_texts

train_texts = converting_texts(train_texts)
test_texts = converting_texts(test_texts)

I need to tokenize my text and padding sequences before modeling my data. I will use Keras proprocessing tools for this.

In [ ]:
max_feat= 12000 #seting max features to define max number of tokenizer words

tokenizer = Tokenizer(num_words=max_feat)
tokenizer.fit_on_texts(train_texts)
# updates internal vocabulary based on a list of texts
# in the case where texts contains lists, we assume each entry of the lists to be a token
# required before using texts_to_sequences or texts_to_matrix

train_texts = tokenizer.texts_to_sequences(train_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)
# transforms each text in texts to a sequence of integers
# Only top num_words-1 most frequent words will be taken into account
# Only words known by the tokenizer will be taken into account

To use batches productively, I need to turn my sequences to same lenght. I prefer to set everything to maximum lenght of the longest sentence in train data.

In [ ]:
max_len = max(len(train_ex) for train_ex in train_texts) #setting the max length

# using pad_sequence tool from Keras
# transforms a list of sequences to into a 2D Numpy array of shape
# the maxlen argument for the length of the longest sequence in the list
train_texts = pad_sequences(train_texts, maxlen=max_len)
test_texts = pad_sequences(test_texts, maxlen=max_len)

## Building Model

In this simple model, convolutional neural nets were used with 64 embedding dimension. 3-convolutional layers used, first two have batch normalization and maximum pooling arguments. The last one has glabal maximum pooling. Results were passed to a dense layer and output for prediction.

Batch normalizations normalize and scale inputs or activations by reducing the amount what the hidden unit values shift around. Max Pool downsamples the input representation by taking the maximum value over the window defined by pool size.

In [ ]:
def build_model():
    sequences = layers.Input(shape=(max_len,))
    embedded = layers.Embedding(max_feat, 64)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model

model = build_model()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


# Fitting Model to My Pre-processed Data

In [ ]:
model.fit(
    train_texts,
    train_target,
    batch_size=128,
    epochs=2,
    validation_data=(test_texts, test_target) )

Train on 63983 samples, validate on 15996 samples
Epoch 1/2
63983/63983 [==============================]63983/63983 [==============================] - 963s 15ms/step - loss: 0.2943 - binary_accuracy: 0.9136 - val_loss: 0.2673 - val_binary_accuracy: 0.9155

Epoch 2/2
63983/63983 [==============================]63983/63983 [==============================] - 1005s 16ms/step - loss: 0.2185 - binary_accuracy: 0.9197 - val_loss: 0.2632 - val_binary_accuracy: 0.9175



This model gives me 0.263 loss value and 0.92% accuracy on validation data.

# Recurrent Neural Net Model

For RNN model layers, I have embedding layer and also I used GRU layers which followed by 2 dense layers.

In [ ]:
def build_rnn_model():
    sequences = layers.Input(shape=(max_len,))
    embedded = layers.Embedding(max_feat, 64)(sequences)
    x = layers.GRU(128, return_sequences=True)(embedded)
    x = layers.GRU(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model

rnn_model = build_rnn_model()

## Fitting Model to My Data

In [ ]:
rnn_model.fit(
    train_texts,
    train_target,
    batch_size=128,
    epochs=1,
    validation_data=(test_texts, test_target) )

Train on 63983 samples, validate on 15996 samples
Epoch 1/1
63983/63983 [==============================]63983/63983 [==============================] - 7834s 122ms/step - loss: 0.2611 - binary_accuracy: 0.9169 - val_loss: 0.2110 - val_binary_accuracy: 0.9203



Even for one epoch this takes too much time, so I opened this notebook in Google Colab and tried the epochs=2 version there and pasted results here.

In [ ]:
# This cell was runned in Colab
# rnn_model.fit(
#     train_texts,
#     train_target,
#     batch_size=128,
#     epochs=2,
#     validation_data=(test_texts, test_target) )

Results for 2 epoches;

- loss: 0.1623
- acc: 0.9371
- val loss: 0.1615
- val acc: 0.9377

I will compare 5 Keras results in next notebook as total. But from here, I found RNN model more accurate.